In [1]:
import sys
sys.path.append("..")

#import jax
#jax.config.update("jax_enable_x64", True)

from utils import *
from model_tn import *
from keras_utils import *
from jax_utils import *

keras.mixed_precision.set_global_policy("mixed_float16")
#keras.mixed_precision.set_global_policy("float32")
#keras.mixed_precision.set_global_policy("float64")

In [2]:
class TSCANToEnd(keras.Model):
    
    def __init__(self, model):
        super().__init__()
        self.inner = model
    
    def call(self, x, training=None):
        return self.inner((x, ops.mean(x, axis=(1, ), keepdims=True)), training=training)

model = TSCANToEnd(TSCAN(TN=True))
model.compile(optimizer='adamw', loss='mse')
model(np.random.random((4, 160, 36, 36, 3)));
model.inner.summary()

Model: "tscan"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (640, 34, 34, 32)      │        10,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (640, 15, 15, 64)      │        55,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_2 (Sequential)       │ (4, 34, 34, 32)        │        10,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_3 (Sequential)       │ (4, 15, 15, 64)        │        55,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_4 (Sequential)       │ (4, 34, 34, 1)         │            33 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_5 (Sequential)       │ (4, 15, 15, 1)         │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_6 (Sequential)       │ (640, 17, 17, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_7 (Sequential)       │ (640, 1)               │       401,665 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 532,899 (2.03 MB)

 Trainable params: 532,899 (2.03 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
batch_size = 128

tape = "/root/ssd_cache/rppg_training_data/mmpd_160x36x36_all"

train_tape = load_datatape(tape, fold='train', batch=batch_size)
valid_tape = load_datatape(tape, fold='val', extended_hr='False', batch=batch_size)

#train_rlap = DatatapeMonitor(train_rlap)
training_set, validation_set = KerasDataset(train_tape), KerasDataset(valid_tape)

training_set = training_set.apply_fn(compress_aug)

mmpd_160x36x36_all             datatape has been loaded.     7552 items total. fold=train selected
mmpd_160x36x36_all             datatape has been loaded.     1664 items total. fold=val&extended_hr=False selected


In [4]:
#model.fit(training_set, validation_data=validation_set, epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/tscantn_mmpd.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)])
train(model, training_set, validation_set, epochs=20, check_point_path='../weights/tscantn_mmpd.weights.h5')
model.load_weights('../weights/tscantn_mmpd.weights.h5')

Training:   0%|          | 0/59 [00:00<?, ?it/s]

Validating:   0%|          | 0/13 [00:00<?, ?it/s]

Epoch   1:  Training loss: 1.126	Validation loss:0.9441	


Training:   0%|          | 0/59 [00:00<?, ?it/s]

Validating:   0%|          | 0/13 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/13 [00:00<?, ?it/s]

Epoch  19:  Training loss:0.8709	Validation loss:  0.87	


Training:   0%|          | 0/59 [00:00<?, ?it/s]

Validating:   0%|          | 0/13 [00:00<?, ?it/s]

Epoch  20:  Training loss:0.8686	Validation loss:0.8696	


In [5]:
eval_on_dataset(dataset_H5_rlap, pmodel(model), 160, (36, 36), step=4, batch=16, save='../results/TSCANtn_MMPD_RLAP.h5', scenes=['R1', 'R2', 'R3', 'R4'])
get_metrics('../results/TSCANtn_MMPD_RLAP.h5')

  0%|          | 0/220 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,2.797±0.111,8.374±2.29,3.725±0.155,0.75778
Whole,1.401±0.159,2.74±1.274,1.841±0.206,0.96473


In [6]:
eval_on_dataset(dataset_H5_cohface, pmodel(model), 160, (36, 36), step=4, batch=16, save='../results/TSCANtn_MMPD_COHFACE.h5', fps=30)
get_metrics('../results/TSCANtn_MMPD_COHFACE.h5')

  0%|          | 0/164 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,6.011±0.272,13.104±3.598,8.209±0.367,0.54373
Whole,4.253±0.482,7.496±3.61,5.734±0.606,0.78313


In [7]:
eval_on_dataset(dataset_H5_pure, pmodel(model), 160, (36, 36), step=4, batch=16, save='../results/TSCANtn_MMPD_PURE.h5')
get_metrics('../results/TSCANtn_MMPD_PURE.h5') 

  0%|          | 0/59 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,1.792±0.235,6.738±3.603,2.732±0.363,0.95669
Whole,1.078±0.393,3.204±2.561,1.565±0.644,0.99047


In [8]:
eval_on_dataset(dataset_H5_ubfc_rppg2, pmodel(model), 160, (36, 36), step=4, batch=16, save='../results/TSCANtn_MMPD_UBFC.h5')
get_metrics('../results/TSCANtn_MMPD_UBFC.h5')

  0%|          | 0/42 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,1.387±0.092,2.525±1.102,1.435±0.096,0.98891
Whole,0.519±0.081,0.741±0.43,0.53±0.078,0.99911
